In [1]:
import sys
import numpy as np
import string

from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation

LAYER_COUNT = 3
HIDDEN_LAYERS_DIM = 500

Using TensorFlow backend.


In [2]:
# generic vocabulary
characters = list(string.printable)
characters.remove('\x0b')
characters.remove('\x0c')

VOCABULARY_SIZE = len(characters)
characters_to_ix = {c:i for i,c in enumerate(characters)}
print("vocabulary len = %d" % VOCABULARY_SIZE)
print(characters)

vocabulary len = 98
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' ', '\t', '\n', '\r']


In [3]:
test_model = Sequential()
for i in range(LAYER_COUNT):
    test_model.add(
            LSTM(
                HIDDEN_LAYERS_DIM, 
                return_sequences=True if (i!=(LAYER_COUNT-1)) else False,
                batch_input_shape=(1, 1, VOCABULARY_SIZE),
                stateful=True
            )
        )
test_model.add(Dense(VOCABULARY_SIZE))
test_model.add(Activation('softmax'))
test_model.compile(loss='categorical_crossentropy', optimizer="adam")

In [4]:
test_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (1, 1, 500)               1198000   
_________________________________________________________________
lstm_2 (LSTM)                (1, 1, 500)               2002000   
_________________________________________________________________
lstm_3 (LSTM)                (1, 500)                  2002000   
_________________________________________________________________
dense_1 (Dense)              (1, 98)                   49098     
_________________________________________________________________
activation_1 (Activation)    (1, 98)                   0         
Total params: 5,251,098
Trainable params: 5,251,098
Non-trainable params: 0
_________________________________________________________________


In [5]:
test_model.load_weights(
    "4-gpu_BS-100_3-500_dp0.20_60S_epoch01-loss2.9326-val-loss2.4937_weights"
)

In [12]:
test_model.load_weights(
    "4-gpu_BS-100_3-500_dp0.20_60S_epoch01-loss1.7700-val-loss1.3981_weights"
)

In [6]:
# May 25
test_model.load_weights(
    "4-gpu_BS-80_3-500_dp0.20_60S_epoch01-loss1.4302-val-loss1.2292_weights"
)

In [28]:
def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array"""
    # from fchollet/keras
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def predict_next_char(model, current_char, diversity=1.0):
    """Predict the next character from the current one"""
    x = np.zeros((1, 1, VOCABULARY_SIZE))
    x[:,:,characters_to_ix[current_char]] = 1
    y = model.predict(x, batch_size=1)
    next_char_ix = sample(y[0,:], temperature=diversity)
    next_char = characters[next_char_ix]
    return next_char

def generate_text(model, seed="I am", count=100):
    """Generate characters from a given seed"""
    model.reset_states()
    for s in seed[:-1]:
        next_char = predict_next_char(model, s)
    current_char = seed[-1]

    sys.stdout.write("["+seed+"]")
    
    for i in range(count - len(seed)):
        next_char = predict_next_char(model, current_char, diversity=0.5)
        current_char = next_char
        sys.stdout.write(next_char)
    print("...\n")

In [25]:
sentence = "In a stunning reversal just a day after canceling his meeting with Kim Jong-un,"

In [27]:
for i in range(2):
    generate_text(
        test_model,
        seed=sentence
    )

[In a stunning reversal just a day after canceling his meeting with Kim Jong-un,] and seemed to be trouble and start conversters about that had to be a very closer from the mazing eyes. It's in the top of the stone chance. ... and all that through the confised again. . . he could be all right that was much to see him everything for that way ..."

"What?" said Harry frantly, "will have a start of the staircase at the stone champions and there was the part of this about the moment, that thing he's n...

[In a stunning reversal just a day after canceling his meeting with Kim Jong-un,] who was still sending a moment with him in the castle, and started to the table carriage from the stone and started to the whole possible of the castle. "I know what we has really see it and got to all read who and try conversation. Professor McGonagall - the first task was starting to the bring and for a face at all. . . the Whole I thought he had some done to the students ... he said and trember. . . but h